In [1]:
import pandas as pd
import numpy as np
import pymysql
from datetime import datetime, timedelta
import warnings
import requests
import json
import fire
warnings.filterwarnings('ignore')

## Datos a comparar

In [ ]:
data_perdida_cosecha = pd.read_excel("../../matriz_modelo_riesgo/data/240531 PERDIDA DE COSECHA.xlsb", sheet_name = 'COLOCACION', header = 1)
data_perdida_cosecha.columns = [k.lower().replace(" ", "_") for k in data_perdida_cosecha.columns]
data_perdida_cosecha.rename(columns={"id_dist.":"id_distribuidor"}, inplace = True)

## extracción de SQL

In [2]:
sql_demo = f"""
select * from (select 
    id as 'APPLICANT_ID', 
    monto_mensual as ingreso_mensual,
    gastos_familiares,
    otros_ingresos,
    replace(replace(replace(ingreso_neto,"$",""), ",",""), ".00","") as ingreso_neto,
    dependientes,
    ventas,
    FLOOR(DATEDIFF(CURDATE(), fecha_nacimiento) / 365.25) AS edad,
    monto_mensual,
    compras,
    delegacion,
    delegacion_negocio,
    escolaridad,
    estado,
    estado_civil,
    estado_negocio,
    sexo,
    sucursal,
    tipo_negocio,
    tipo_vivienda
from cc_prospectos_demograficos

UNION

select 
    id as 'APPLICANT_ID', 
    ingreso_mensual,
    gastos_familiares,
    otros_ingresos,
    replace(replace(replace(ingreso_neto,"$",""), ",",""), ".00","") as ingreso_neto,
    dependientes,
    ventas,
    FLOOR(DATEDIFF(CURDATE(), fecha_nacimiento) / 365.25) AS edad,
    monto_mensual_venta,
    compras,
    delegacion,
    delegacion_negocio,
    escolaridad,
    estado,
    estado_civil,
    estado_negocio,
    sexo,
    sucursal,
    tipo_negocio,
    tipo_vivienda
from demograficos_distribuidores) as pen
where pen.APPLICANT_ID in {ids}
"""

NameError: name 'ids' is not defined

In [ ]:
cnx = pymysql.connect(host='159.89.90.197', port = 3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                               charset='utf8',db = 'volana')
cursor = cnx.cursor()
cursor.execute(sql_demo)
data_demo = cursor.fetchall()
data_demo = pd.DataFrame(data_demo, columns=[str(i[0]).upper() for i in cursor.description])
cnx.close()